# Revised case normalization for KSSG Linth 2019

Note:

- All 15 of 15 cases from the revised data could be matched with the BFS DB data of KSSG Linth 2019. 



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Linth_Toggenburg_SRRWS_2019.xlsx'),
         'KSSG Spital Linth',
         '2019',
         'Änderungen_Spital_Linth_2019')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Linth_Toggenburg_SRRWS_2019.xlsx', hospital_name_db='KSSG Spital Linth', year='2019', sheet='Änderungen_Spital_Linth_2019')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 09:19:55.258 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 15 cases for KSSG Spital Linth 2019
2022-11-21 09:19:55.263 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
tranche                                string
datum                                  string
case_id                                object
patient_id                             object
kkik                                   string
gender                                 string
age_years                               int64
bfs_code                               string
duration_of_stay                        int64
pflegetage neu                         string
pccl                                    int64
pccl neu                               string
old_pd                                 string
primary_diagnosis                      string
added_icds                             string
removed_icds                          

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,3108134918,DDB3BDB39DE3F695,M,49,7,3108134918,A46,A46,M100,3,J64C,[K746],[],[],[]
1,3108136936,1C6A9D7FAA9A7548,W,89,5,3108136936,K226,K226,M100,2,G46C,[N184],[D6833],[],[]
2,3108141419,69231F9C61534D42,M,69,10,3108141419,C183,C183,M100,2,G60B,[E43],[],[],[]
3,3150158985,29F13DDB90E1976D,W,70,2,3150158985,K922,K922,M100,3,G48C,"[D62, D684]",[D500],[],[]
4,3150164388,11721C6EB40D5BB5,M,78,5,3150164388,K250,K250,M100,2,G46C,[D62],[],[],[]
5,3108135794,5B6393F1E0401199,M,83,6,3108135794,K565,K565,M200,3,G18B,[J91],[],[],[]
6,3150163534,8F567DBC59007A54,W,77,40,3150163534,T818,T818,M100,3,X62B,[I5001],[],[],[]
7,3108132452,DA9064B3A6941B98,M,84,9,3108132452,T846,T846,M200,2,I12C,[E440],[],[890A32],[]
8,3108136823,5FBF55310D4ED16F,W,97,5,3108136823,S701,M6285,M200,3,X60C,[F058],[],[],[]
9,3108139676,8BADDE1F77393EE1,W,70,5,3108139676,S501,T7960,M200,3,X06B,"[E871, N938]",[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-21 09:19:58.995 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 5106 rows from the DB, for the hospital 'KSSG Spital Linth' in 2019


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,527510,3150163156,T814,"[T814, Y849, B957, I1190, Z950, ...]",867A3E:L:20191112,[],M,87,0,0,15,0,01,2019-11-12,0,06,2019-11-27
1,528046,3150158985,K922,"[E1190, I1090, E039, E788, R571, ...]",990410::20190618,"[451310::20190619, 990410::20190620]",W,70,0,0,2,0,01,2019-06-18,0,06,2019-06-20
2,528282,3108132184,N1783,"[N183, E86, A099, I2513, I1101, ...]",990411::20190322,[],M,72,0,0,12,0,01,2019-03-22,0,00,2019-04-03
3,528434,3108134918,A46,"[I5014, I1100, I480, Z921, F102, ...]",,[],M,49,0,0,7,0,01,2019-05-31,0,00,2019-06-07
4,528903,3150164388,K250,"[D6835, T810, Y849, N1781, N182, ...]",444399::20191216,"[451310::20191218, 990410::20191214]",M,78,0,0,5,0,01,2019-12-14,0,00,2019-12-19


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 09:19:59.482 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 15 cases ...
2022-11-21 09:20:00.542 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 15 cases into: 15 revisions, 150 diagnoses rows, 55 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,527510,T01C,1.50,1.50,2,2022-12-31
1,528046,G46B,1.33,0.92,4,2022-12-31
2,528282,L60C,1.29,1.29,4,2022-12-31
3,528434,J64A,1.42,1.42,4,2022-12-31
4,528903,G46B,1.33,1.33,3,2022-12-31
5,529021,G04Z,3.33,3.33,4,2022-12-31
6,529366,I72Z,1.13,1.13,4,2022-12-31
7,529789,G48C,0.97,0.97,3,2022-12-31
8,530594,X62A,1.53,5.34,4,2022-12-31
9,530657,I03A,3.71,3.71,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 09:20:00.563 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 15 cases into the 'Revisions' table ...
2022-11-21 09:20:03.470 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 15 cases into the 'Revisions' table
2022-11-21 09:20:03.474 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 150 rows into the 'Diagnoses' table ...
2022-11-21 09:20:05.633 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 150 rows into the 'Diagnoses' table
2022-11-21 09:20:05.636 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 55 rows into the 'Procedures' table ...
2022-11-21 09:20:07.263 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 55 rows into the 'Procedures' table
